# Sentiment Analysis POC

* This notebook is a scratch pad, for sentiment analysis POC. 
* The idea is to try out few pre-trained sentiment analysis models and see which one works for our use case. 

## Installations

In [1]:
# ## installing required libraries
! pip install beautifulsoup4
! pip install pandas
! pip install numpy
! pip install plotly
! pip install nbformat
! pip install ipykernel
! pip install matplotlip
! pip install wordcloud
! pip install gensim
! pip install pyLDAvis
! pip install nltk
! pip install -U pip setuptools wheel
! pip install -U spacy
! python -m spacy download en_core_web_trf 
! python -m spacy download en_core_web_md
! pip install joblib
! pip install tqdm


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 42.8/42.8 MB 29.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Importing Data

In [2]:
## lets load 
import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

import re
import string
from bs4 import BeautifulSoup

import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
from pprint import pprint

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as io

# loading library
import pickle

from joblib import dump, load

from tqdm.auto import tqdm

import os
import sys
sys.path.insert(0, os.path.abspath('../utils'))

## importing custom modules
import common_utils
import gensim_utils
import sklearn_utils

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\workspace\Aletheia\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\workspace\Aletheia\env\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gaura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] D

## Reading Data

In [3]:
## reading manaully scrapped data
data = pd.read_csv('../data/scrapped_fox_data_clean.csv')
print(data.shape)

(3972, 12)


In [4]:
def preprocess_text(text):
     text = common_utils.clean_html(text)
     text = common_utils.lower_case(text)
     text = common_utils.remove_line_breaks(text)
     text = common_utils.remove_punctuation(text)
     text = common_utils.remove_numbers(text)
     text = common_utils.remove_extra_spaces(text)
     # text = remove_stopwords(text)
     return text

data["clean_text"] = data["text"].progress_apply(preprocess_text)

processing: 100%|██████████| 3972/3972 [00:01<00:00, 2557.60it/s]


In [5]:
data.head()["clean_text"]

0    former governor and first term democratic sen ...
1    president biden urged democrats on wednesday t...
2    the famous naked cowboy in new york citys time...
3    liberal groups in wisconsin seeking to change ...
4    texas gubernatorial nominee beto o’rourke is a...
Name: clean_text, dtype: object

## Sentiment Analysis Using VADER
(Valence Aware Dictionary and sEntiment Reasoner)

##### Notes
* Uses Bag of words approach
* Gives +tive, -tive or neutral values to each of the words in the sentence and then gives combined value of that to tell us whether the sentence is positive, negative or neutral
* Does not account for relationship between words :(

In [6]:
from nltk.sentiment import SentimentIntensityAnalyzer


In [7]:
example = data["text"][0]
example

'Former governor and first term Democratic Sen. Maggie Hassan of New Hampshire and Republican challenger Don Bolduc took aim at each other over inflation, abortion, national security, the border crisis, election denialism, and many more issues in their third and final debate in their crucial battleground state race that’s among a handful across the country that will likely determine if the GOP wins back the Senate majority. But ahead of the verbal crossfire on the debate stage, Bolduc – a former Army general who served ten tours of duty in the war in Afghanistan – was allegedly assaulted as he arrived at the debate site at Saint Anselm College’s New Hampshire Institute of Politics on Wednesday evening. According to the Bolduc campaign, a bystander standing in the crowd outside the debate site took a swing at the former general as he arrived. The campaign says Bolduc was slightly grazed but not injured.&nbsp; Rick Wiley of the Bolduc campaign tells Fox News the person who threw the punc

In [9]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [8]:
sia = SentimentIntensityAnalyzer()

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\gaura/nltk_data'
    - 'd:\\workspace\\Aletheia\\env\\nltk_data'
    - 'd:\\workspace\\Aletheia\\env\\share\\nltk_data'
    - 'd:\\workspace\\Aletheia\\env\\lib\\nltk_data'
    - 'C:\\Users\\gaura\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [23]:
sia.polarity_scores(example)

NameError: name 'sia' is not defined